<a href="https://colab.research.google.com/github/g-aditi/moral-compass/blob/Nihaarika/Moral_Compass_VectorDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.5 MB/s eta 0:00:00


In [2]:
import PyPDF2
import pandas as pd
import numpy as np
import os
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
def pdf_to_text(pdf_data_directory, txt_data_directory):
    # Converting pdf version paper to txt file

    if not os.path.exists(txt_data_directory):
        os.makedirs(txt_data_directory)

    for pdf_filename in os.listdir(pdf_data_directory):

        if pdf_filename.endswith(".pdf"):

            pdf_path = os.path.join(pdf_data_directory, pdf_filename)
            #checkpoint
            print (pdf_filename)
            txt_filename = os.path.splitext(pdf_filename)[0] + ".txt"
            txt_path = os.path.join(txt_data_directory, txt_filename)

            with open(pdf_path, "rb") as pdf_file:
                pdf_reader = PyPDF2.PdfReader(pdf_file)
                text = ""
                for page_num in range(len(pdf_reader.pages)):
                    page = pdf_reader.pages[page_num]
                    text += page.extract_text()

                with open(txt_path, "w", encoding="utf-8") as txt_file:
                    txt_file.write(text)

pdf_data_directory = "/content/sample_data/MoralCompass_Papers_VDB_pdf"
txt_data_directory = "/content/sample_data/MoralCompass_Papers_VDB_txt"
pdf_to_text(pdf_data_directory, txt_data_directory)



466535.pdf
Human Subject Regulations Decision Charts_ Pre-2018 Requirements _ HHS.gov.pdf
Subpart D — Additional Protections for Children Involved _ HHS.gov.pdf
32843821.pdf
Single IRB Exception Determinations _ HHS.gov.pdf
Exemptions (2018 Requirements) _ HHS.gov.pdf
Annotated Comparison of the Pre-2018 Common Rule with _ HHS.gov.pdf
Revised Common Rule _ HHS.gov.pdf
Subpart E — Registration of Institutional Review Boards _ HHS.gov.pdf
Terminology _ HHS.gov.pdf
October 8, 2020_ Exception to the Single IRB Review Requirements _ HHS.gov.pdf
OHRP Expedited Review Categories (1998) _ HHS.gov.pdf
Procedures-for-HSR-2022.pdf
Pre-2018 Requirements _ HHS.gov.pdf
the-belmont-report-508c_FINAL.pdf
eCFR __ 45 CFR Part 46 (July 19, 2018) -- Protection of Human Subjects.pdf
Subpart C — Additional Protections Pertaining to Biomedical and B _ HHS.gov.pdf
November 21, 2019_ Determination of Exception for Certain HHS _ HHS.gov.pdf
Excerpts from the January 19, 2017 Revised Common Rule Preamble _ HHS.g

In [7]:
# training a document vectorizer model

txt_data_directory = "/content/sample_data/MoralCompass_Papers_VDB_txt"
txt_file_paths = [os.path.join(txt_data_directory, file) for file in os.listdir(txt_data_directory) if file.endswith('.txt')]
tagged_data = []
for i, txt_file_path in enumerate(txt_file_paths):
    with open(txt_file_path, 'r', encoding='utf-8') as file:
         text = file.read()
         words = word_tokenize(text)
         words = [word.lower() for word in words]
         tagged_data.append(TaggedDocument(words, tags=['doc_' + str(i)]))
model = Doc2Vec(vector_size=20, epochs=300)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

document_vectors = [model.dv['doc_' + str(i)] for i in range(len(tagged_data))]
#checkpoint
print (document_vectors)

[array([-5.5733156 , -5.109174  , -0.8917723 ,  2.4881642 ,  3.1329553 ,
       -1.2328875 , -1.9686515 , -6.134225  ,  0.0560947 , -2.5560877 ,
        4.48137   , -0.48160103, -2.0046444 ,  1.6887807 ,  0.0429637 ,
        2.192132  ,  0.74103373,  1.3238579 , -6.109742  , -1.253444  ],
      dtype=float32), array([-3.1306145 ,  0.09271465, -2.837216  , -0.23501016,  2.9095986 ,
        0.7481839 , -2.8151186 , -3.9100726 , -0.5079803 ,  0.09291416,
        6.1484165 , -0.5595097 , -3.3624754 , -3.1028783 , -0.17693613,
        3.2351182 ,  3.3627305 ,  2.2587385 , -4.956069  ,  1.3461589 ],
      dtype=float32), array([-2.789852  , -1.8020542 , -0.6309695 ,  1.2294955 ,  3.7705462 ,
       -2.071355  , -3.47569   , -0.08491134,  2.0683408 ,  2.089196  ,
        3.9341106 ,  2.2231023 , -3.123347  , -2.5251372 , -0.7414659 ,
        4.404957  ,  1.8728417 ,  1.2577014 , -3.9513881 ,  2.6858404 ],
      dtype=float32), array([-0.35748017,  3.5727274 ,  3.1266825 ,  3.7969193 ,  3.8740

In [8]:
!pip install pandas sentence-transformers pandarallel faiss-cpu



  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.4 MB/s eta 0:00:00
  Created wheel for pandarallel: filename=pandarallel-1.6.5-py3-none-any.whl size=16675 sha256=ae1ce90bccd605323f7009cdcac5f974cc03f2e32fd01234eb8ed9a0dc3d5a1e
  Stored in directory: /root/.cache/pip/wheels/50/4f/1e/34e057bb868842209f1623f195b74fd7eda229308a7352d47f
Successfully built pandarallel


In [9]:
import faiss

In [10]:
vector_dim = len(document_vectors[0])
#checkpoint
print (vector_dim)
index = faiss.IndexFlatL2(vector_dim)
vectors_np = np.array(document_vectors).astype('float32')

20
